In [1]:
from utils.schedule_utils import load_data_split,table_to_str,table_to_str_sql,find_intersection_and_add_row_id,Prepare_Data_for_Operator_Sequence,format_document,batch_rerank_scores,ROLLBACK,merge_clean_and_format_df_dict,retrieve_rows_by_subtables,process_error_analysis_list
import pickle
import numpy as np
import pandas as pd
import os
import json
from FlagEmbedding import FlagReranker
from tqdm import tqdm
from typing import Dict, Any
from utils.evaluator import Evaluator
from utils.prompt_generate import build_wikitq_prompt_from_df,evaluate_predictions,filter_dataframe_from_responses,fix_sql_query,match_subtables,retrieve_rows_by_subtables,build_tab_fact_prompt_from_df
from utils.async_llm import infer_prompts

In [3]:
### 读取必要文件 （step 2）
dataset_name = 'wikitq'
split = 'test'
dataset = load_data_split(dataset_name,split)
wikitq_df_processed = np.load(f'datasets/schedule_test/{dataset_name}/wikitq_df_processed.npy',allow_pickle=True).item()
wikitq_df = pd.DataFrame(dataset)

In [4]:
result = pd.read_csv(f'../datasets/pipeline/wikitq/wikitq_test_rewrite_RAG_qa_output_4B.csv',index_col=0)
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, result, dataset)

  0%|          | 0/4344 [00:00<?, ?it/s]

Correct Samples: 2976; Total Samples: 4344
Accuracy: 68.51


### RAG Ablation Test for Query Rewrite


In [14]:
prompt_list = []
for index in range(len(dataset)):
    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/rewrite_query.txt',processed=True)
    prompt_list.append(prompt)
wikitq_df['instruction'] = prompt_list

In [17]:
wikitq_df.to_csv('../datasets/pipeline/wikitq/wikitq_test_rewrite.csv')

In [7]:
rewrite_result = pd.read_csv('../datasets/pipeline/wikitq/wikitq_test_rewrite_output_4B.csv',index_col=0)

In [8]:
dataset[4]['question']

'what is the number of 1st place finishes across all events?'

In [10]:
rewrite_query_list[4]

"[GENERAL]: Count the total number of times a rider achieved a 1st place finish (placing = 1) across all events and competitions.  \n[BALANCED]: Count rows where 'placing' is '1' in the 'placing' column to determine the total number of first-place finishes.  \n[SPECIFIC]: count first place finishes placing 1 event competition location country rider nationality victoria pendleton gbr jason kenny gbr ross edgar gbr jamie staff gbr chris hoy gbr"

In [12]:
np.save('../schedule_pipeline/datasets/schedule_test/wikitq/rewrite_query_4B.npy',rewrite_query_list)

In [ ]:
python Hybrid_Retrieve_Update.py --model_path /data/workspace/yanmy/models/bge-m3 --dataset_name wikitq --split test --index_path datasets/schedule_test/wikitq/RAG_index_all.npy --output_path datasets/schedule_test/wikitq/Hybrid_Retrieve_output_update.npy --max_rows 50 --max_cols 10 --processed_df_path datasets/schedule_test/wikitq_7B/wikitq_df_processed.npy --rewrite_query_path datasets/schedule_test/wikitq/rewrite_query_4B.npy

In [13]:
np.save('datasets/schedule_test/wikitq/RAG_index_all.npy',[i for i in range(len(dataset))])

In [14]:
RAG = np.load('datasets/schedule_test/wikitq/Hybrid_Retrieve_output_update.npy',allow_pickle=True).item()

In [9]:
rewrite_query_list = []
for index in range(len(dataset)):
    rewritten_query = eval(rewrite_result.iloc[index,-1])[0]
    # dataset[index]['question'] = rewritten_query
    rewrite_query_list.append(rewritten_query)
    # wikitq_df.at[index,'rewritten_query'] = rewritten_query

In [23]:
processed_table = np.load('../datasets/check/wikitq_test_processed_table.npy', allow_pickle=True).item()

In [15]:
prompt_list = []
for index in range(len(dataset)):
    rewritten_query = eval(rewrite_result.iloc[index,-1])[0]
    # sequence = Check_Model_Data_Sequence[index]['Sequence']
    prompt = build_wikitq_prompt_from_df(dataset,RAG[index],index,template_path='../prompts/text_reason_wtq.txt',processed=True)
    prompt = prompt.replace(dataset[index]['question'], f"[ORIGINAL]: {dataset[index]['question']}  \n" + rewritten_query)
    # if sequence==[] or sequence.__contains__('Execute_SQL'):
    # evidence = table_to_str_sql(processed_table['Execute_SQL'][index])
    # prompt = prompt + evidence
    prompt_list.append(prompt)
wikitq_df['instruction'] = prompt_list

In [16]:
wikitq_df.to_csv('  ')

In [18]:
def evaluate_predictions(dataset_name, df, dataset):
    acc =0
    error_index = []
    format_error_index = []
    # dataset_name = 'wikitq'
    dataset_df_output = df
    outputs = dataset_df_output.to_dict(orient='index')
    # Iterate through each output and calculate the accuracy
    count = 0
    for i in tqdm(outputs.keys()):
        count += 1
        output = outputs[i]
        pred_answer = None

        # --- Prediction Parsing Logic ---
        if dataset_name == 'wikitq':
            try:
                # Attempt to parse different possible output formats
                raw_output = eval(output['predict'])[0]
                if 'the answer is: ' in raw_output:
                    pred_answer_all = raw_output.split('the answer is: ')[1]
                elif 'Final Answer: ' in raw_output:
                    pred_answer_all = raw_output.split('Final Answer: ')[1]
                else:
                    raise ValueError("Unrecognized format")
                
                # Extract the answer, which is often quoted
                pred_answer = pred_answer_all.split('"')[1:2]

            except Exception as e:
                # If parsing fails, log it as a format error
                # print(f'Format error in index {i}: {e}')
                format_error_index.append(i)
                pred_answer = [''] # Assign an empty prediction
                pred_answer_all = ''
                # print(pred_answer)

        elif dataset_name == 'tab_fact':
            # For TabFact, check for 'true' or 'false' (case-insensitive)
            # if isinstance(output['predict'], str) and 'true' in output['predict'].lower():
            try:
                pred_answer_str = eval(output['predict'])[0].lower().split('statement is: ')[1].replace(" ", "")
            except:
                pred_answer_str = eval(output['predict'])[0]
            if 'false' in pred_answer_str:
                pred_answer = [0]
            else:
                pred_answer = [1]

            # gold_answer = result_df[str(i)]['ori_data_item']['answer_text']
        gold_answer = dataset[i]['answer_text']
        # Score is either 1 or 0
        # print(pred_answer)
        score = Evaluator().evaluate(
        pred_answer,
        gold_answer,
        dataset=dataset_name,
        question=output['question']
        )
        if dataset_name=='wikitq':
            if score == False and isinstance(pred_answer_all, str):
                score = Evaluator().evaluate(
                    pred_answer_all.split(','),
                    gold_answer,
                    dataset=dataset_name,
                    question=output.get('question', '') # Safely get question if available
                )
        acc += score
        if score != 1:
            error_index.append(i)
            # if score!=1:
            #     print(f'The prediction is {pred_answer}, while the ground truth is {gold_answer} for sample {i}.')
        # except:
        #     print(i)
        #     pass
    final_accuracy = 100 * acc / count if count > 0 else 0
    print(f"Correct Samples: {acc}; Total Samples: {count}")
    print(f"Accuracy: {100*acc/count:.2f}")
    return final_accuracy, error_index, format_error_index

In [43]:
dataset_name

'wikitq'

In [ ]:
result = pd.read_csv('../datasets/pipeline/wikitq/wikitq_test_rewrite_qa_output_4B.csv',index_col=0)
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, result, dataset)

In [22]:
result = pd.read_csv('../datasets/wikitq_test_output_4B.csv',index_col=0)
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, result, dataset)

100%|██████████| 4344/4344 [01:23<00:00, 51.83it/s]

Correct Samples: 3231; Total Samples: 4344
Accuracy: 74.38


In [26]:
result = pd.read_csv('../datasets/pipeline/wikitq/wikitq_test_rewrite_qa_output_4B.csv',index_col=0)
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, result, dataset)

100%|██████████| 4344/4344 [01:21<00:00, 53.14it/s]

Correct Samples: 3312; Total Samples: 4344
Accuracy: 76.24


In [25]:
print(result.iloc[31,-2])

You are an expert on table data.
You must use the table data and the additional evidence to answer the given question.

Procedure:
- Divide the main statement into sub-tasks and answer each sub-task
- Based on the answers, check whether the statement is supported by the table

**NOTE** 
Please be extremely careful, pause, make sure all instructions have been followed and only then output the answer

Response Format:
Begin your response with 'Output: ' and always include the following:
- Decompose: Divide the main question into sub-tasks and answer each sub-task
- Final Answer: Strictly output as a short phrase starting by `therefore, the answer is: "AnswerName1", "AnswerName2"...` form, no other form

- Read the question carefully, understand, and return what the question asks.
- Be careful, make sure you have followed all instructions and only then return the output.
```
<input>
table caption: List of spans
/*
col : tramway | year of inauguration
row 0 : 3s aerial tramway | 2004
row 1